In [144]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import KFold
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
np.set_printoptions(suppress=True)  #Suppress Scientific Notation

In [84]:
%run get-ev-scratch-paper.ipynb

,Unnamed: 0,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,0,459.0,-833.0,0.285714,0.714286,1.0
1,1,-110.0,-110.0,0.500000,0.500000,0.0
2,2,-385.0,299.0,0.416667,0.583333,1.0
3,3,-137.0,112.0,0.416667,0.583333,0.0
4,4,-139.0,106.0,0.779412,0.220588,1.0
...,...,...,...,...,...,...
90,90,135.0,-179.0,0.176471,0.823529,1.0
91,91,-323.0,252.0,0.779412,0.220588,0.0
92,92,-227.0,170.0,0.779412,0.220588,0.0
93,93,-141.0,106.0,0.833333,0.166667,1.0



          Number of matches: 95
          Number of bets: 81
          Number of winning bets: 44
          Number of losing bets: 37
          Number of underdog bets: 26
          Number of underdog wins: 8
          Number of underdog losses: 18
          Number of Favorite bets: 55
          Number of favorite wins: 36
          Number of favorite losses: 19
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -438.9048172864855
          Profit per bet: -5.418577991191179
          Profit per match: -4.620050708278795
          
          


In [85]:
df=pd.read_csv("owl-with-odds.csv")

In [86]:
subset = ['t1_odds', 't2_odds']
print(len(df))
df.dropna(subset=subset ,inplace=True)
print(len(df))

df['t1_odds'] = pd.to_numeric(df['t1_odds'], errors='coerce')
df['t2_odds'] = pd.to_numeric(df['t2_odds'], errors='coerce')
df.dropna(subset=subset ,inplace=True)
print(len(df))

651
538
536


In [87]:
#Split the validation set
df_train = df[:-60]

df_test = df[-60:]
print(len(df_test))
print(len(df_train))

60
476


In [88]:
#inclusive features
features = ['corona_virus_isolation', 't1_wins_season',
       't1_losses_season', 't2_wins_season', 't2_losses_season',
       't1_matches_season', 't2_matches_season', 't1_win_percent_season',
       't2_win_percent_season', 't1_wins_alltime', 't1_losses_alltime',
       't2_wins_alltime', 't2_losses_alltime', 't1_matches_alltime',
       't2_matches_alltime', 't1_win_percent_alltime',
       't2_win_percent_alltime', 't1_wins_last_3', 't1_losses_last_3',
       't2_wins_last_3', 't2_losses_last_3', 't1_win_percent_last_3',
       't2_win_percent_last_3', 't1_wins_last_5', 't1_losses_last_5',
       't2_wins_last_5', 't2_losses_last_5', 't1_win_percent_last_5',
       't2_win_percent_last_5', 't1_wins_last_10', 't1_losses_last_10',
       't2_wins_last_10', 't2_losses_last_10', 't1_win_percent_last_10',
       't2_win_percent_last_10', 
        't1_wins_vs_t2', 't1_losses_vs_t2',
       't1_matches_vs_t2', 't1_odds', 't2_odds', 'winner_label']

In [89]:
df_test_filtered = df_test[features].copy()
df_train_filtered = df_train[features].copy()

In [90]:
df_test_filtered.dropna(inplace=True)
df_train_filtered.dropna(inplace=True)

print(df_test_filtered.shape)
print(df_train_filtered.shape)

(60, 41)
(379, 41)


### CANDIDATE 1


In [91]:

model_1 = tree.DecisionTreeClassifier(max_depth=10, min_samples_leaf=3, random_state=75)
features_1 = ['t1_wins_last_3', 't2_matches_alltime', 'winner_label', 't1_odds', 't2_odds']
#FINAL BEST SCORE: 74.52979394547228

In [ ]:
model_2 = RandomForestClassifier(random_state=75)
features_2 = ['t2_wins_last_5', 't1_win_percent_season', 't1_losses_season', 't2_losses_last_5', 't1_wins_vs_t2',
            'winner_label', 't1_odds', 't2_odds']

In [118]:
X=df_train_filtered[features_1].copy()
X_test=df_test_filtered[features_1].copy()
y = X['winner_label']
y_test=X_test['winner_label']

In [119]:
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)

In [120]:
model_1.fit(X, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=75, splitter='best')

In [121]:
probs=model_1.predict_proba(X_test)

In [122]:

X_test_list = np.array(X_test)

X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))



ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])

#display(ev_prepped_df)


get_ev_from_df(ev_prepped_df, True)


          Number of matches: 60
          Number of bets: 56
          Number of winning bets: 29
          Number of losing bets: 27
          Number of underdog bets: 25
          Number of underdog wins: 9
          Number of underdog losses: 16
          Number of Favorite bets: 31
          Number of favorite wins: 20
          Number of favorite losses: 11
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -634.6256471651691
          Profit per bet: -11.332600842235163
          Profit per match: -10.577094119419485
          
          


0

In [123]:
model_2 = RandomForestClassifier(random_state=75)
features_2 = ['t2_wins_last_5', 't1_win_percent_season', 't1_losses_season', 't2_losses_last_5', 't1_wins_vs_t2',
            'winner_label', 't1_odds', 't2_odds']

In [124]:
X=df_train_filtered[features_2].copy()
X_test=df_test_filtered[features_2].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
model_2.fit(X, y)
probs=model_2.predict_proba(X_test)
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)


          Number of matches: 60
          Number of bets: 53
          Number of winning bets: 18
          Number of losing bets: 35
          Number of underdog bets: 33
          Number of underdog wins: 7
          Number of underdog losses: 26
          Number of Favorite bets: 20
          Number of favorite wins: 11
          Number of favorite losses: 9
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -1455.9719218957919
          Profit per bet: -27.47116833765645
          Profit per match: -24.2661986982632
          
          


0

In [125]:
#Model 3
model_3 = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=12, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=75, splitter='best')

features_3 = ['t1_wins_last_3', 't2_matches_alltime', 'winner_label', 't1_odds', 't2_odds']

In [136]:
X=df_train_filtered[features_3].copy()
X_test=df_test_filtered[features_3].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
model_3.fit(X, y)
probs=model_3.predict_proba(X_test)
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)
probs_3 = probs


          Number of matches: 60
          Number of bets: 57
          Number of winning bets: 34
          Number of losing bets: 23
          Number of underdog bets: 26
          Number of underdog wins: 11
          Number of underdog losses: 15
          Number of Favorite bets: 31
          Number of favorite wins: 23
          Number of favorite losses: 8
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: 559.41199353654
          Profit per bet: 9.814245500641052
          Profit per match: 9.323533225609
          
          


In [128]:
#Model 4
model_4 = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=6, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=75, verbose=0,
                       warm_start=False)

features_4=['t2_wins_last_5', 't1_win_percent_season', 't1_losses_season', 't2_losses_last_5', 't1_wins_vs_t2', 'winner_label', 't1_odds', 't2_odds']

In [129]:
X=df_train_filtered[features_4].copy()
X_test=df_test_filtered[features_4].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
model_4.fit(X, y)
probs=model_4.predict_proba(X_test)
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)


          Number of matches: 60
          Number of bets: 50
          Number of winning bets: 17
          Number of losing bets: 33
          Number of underdog bets: 32
          Number of underdog wins: 7
          Number of underdog losses: 25
          Number of Favorite bets: 18
          Number of favorite wins: 10
          Number of favorite losses: 8
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: -1457.8400890870064
          Profit per bet: -29.15680178174013
          Profit per match: -24.297334818116774
          
          


0

In [133]:
#model_5 

model_5 = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.2, loss='deviance', max_depth=16,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=6, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=200,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=75, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

features_5 = ['corona_virus_isolation', 't1_wins_season', 't1_losses_season', 't2_wins_season', 't2_losses_season', 't1_matches_season', 't2_matches_season', 't1_win_percent_season', 't2_win_percent_season', 't1_wins_alltime', 't1_losses_alltime', 't2_wins_alltime', 't2_losses_alltime', 't1_matches_alltime', 't2_matches_alltime', 't1_win_percent_alltime', 't2_win_percent_alltime', 't1_wins_last_3', 't1_losses_last_3', 't2_wins_last_3', 't2_losses_last_3', 't1_win_percent_last_3', 't2_win_percent_last_3', 't1_wins_last_5', 't1_losses_last_5', 't2_wins_last_5', 't2_losses_last_5', 't1_win_percent_last_5', 't2_win_percent_last_5', 't1_wins_last_10', 't1_losses_last_10', 't2_wins_last_10', 't2_losses_last_10', 't1_win_percent_last_10', 't2_win_percent_last_10', 't1_wins_vs_t2', 't1_losses_vs_t2', 't1_matches_vs_t2', 'winner_label', 't1_odds', 't2_odds']

In [137]:
X=df_train_filtered[features_5].copy()
X_test=df_test_filtered[features_5].copy()
y = X['winner_label']
y_test=X_test['winner_label']
X = X.drop('winner_label', axis=1)
X_test = X_test.drop('winner_label', axis=1)
model_5.fit(X, y)
probs=model_5.predict_proba(X_test)
X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], probs[:, 0], probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)
probs_5 = probs


          Number of matches: 60
          Number of bets: 54
          Number of winning bets: 34
          Number of losing bets: 20
          Number of underdog bets: 17
          Number of underdog wins: 8
          Number of underdog losses: 9
          Number of Favorite bets: 37
          Number of favorite wins: 26
          Number of favorite losses: 11
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: 168.08688556683018
          Profit per bet: 3.1127201030894476
          Profit per match: 2.801448092780503
          
          


In [138]:
##We have 2 models returning a profit.  Let's see how an ensemble works....
print(probs_3)
print(probs_5)

[[0.83333333 0.16666667]
 [0.         1.        ]
 [0.16666667 0.83333333]
 [0.75       0.25      ]
 [0.         1.        ]
 [0.         1.        ]
 [0.         1.        ]
 [0.28571429 0.71428571]
 [0.25       0.75      ]
 [0.83333333 0.16666667]
 [0.75       0.25      ]
 [0.83333333 0.16666667]
 [0.28571429 0.71428571]
 [1.         0.        ]
 [0.16666667 0.83333333]
 [1.         0.        ]
 [0.75       0.25      ]
 [1.         0.        ]
 [0.5        0.5       ]
 [0.         1.        ]
 [0.28571429 0.71428571]
 [0.25       0.75      ]
 [0.28571429 0.71428571]
 [0.5        0.5       ]
 [0.         1.        ]
 [0.16666667 0.83333333]
 [1.         0.        ]
 [0.66666667 0.33333333]
 [0.4        0.6       ]
 [0.4        0.6       ]
 [0.         1.        ]
 [0.83333333 0.16666667]
 [0.         1.        ]
 [0.5        0.5       ]
 [0.5        0.5       ]
 [0.83333333 0.16666667]
 [0.         1.        ]
 [0.         1.        ]
 [1.         0.        ]
 [0.         1.        ]


In [145]:
print(probs_5)

[[0.99676961 0.00323039]
 [0.00403209 0.99596791]
 [0.0042296  0.9957704 ]
 [0.98110027 0.01889973]
 [0.04720231 0.95279769]
 [0.0979493  0.9020507 ]
 [0.67945483 0.32054517]
 [0.77810982 0.22189018]
 [0.00434561 0.99565439]
 [0.89954912 0.10045088]
 [0.10707607 0.89292393]
 [0.98308445 0.01691555]
 [0.30812213 0.69187787]
 [0.98744258 0.01255742]
 [0.25702465 0.74297535]
 [0.99944424 0.00055576]
 [0.01072001 0.98927999]
 [0.97720947 0.02279053]
 [0.00904196 0.99095804]
 [0.00595392 0.99404608]
 [0.53121376 0.46878624]
 [0.9586807  0.0413193 ]
 [0.09905414 0.90094586]
 [0.98730397 0.01269603]
 [0.00035286 0.99964714]
 [0.83092216 0.16907784]
 [0.99041418 0.00958582]
 [0.00819728 0.99180272]
 [0.54923941 0.45076059]
 [0.74829485 0.25170515]
 [0.00069157 0.99930843]
 [0.91124983 0.08875017]
 [0.00010008 0.99989992]
 [0.98647937 0.01352063]
 [0.00157112 0.99842888]
 [0.64277766 0.35722234]
 [0.64538549 0.35461451]
 [0.00489741 0.99510259]
 [0.94827819 0.05172181]
 [0.00378079 0.99621921]


In [140]:
ensemble_probs=(probs_3 + probs_5) * .5

In [141]:
ensemble_probs

array([[9.15051472e-01, 8.49485275e-02],
       [2.01604697e-03, 9.97983953e-01],
       [8.54481342e-02, 9.14551866e-01],
       [8.65550135e-01, 1.34449865e-01],
       [2.36011547e-02, 9.76398845e-01],
       [4.89746497e-02, 9.51025350e-01],
       [3.39727414e-01, 6.60272586e-01],
       [5.31912054e-01, 4.68087946e-01],
       [1.27172803e-01, 8.72827197e-01],
       [8.66441229e-01, 1.33558771e-01],
       [4.28538037e-01, 5.71461963e-01],
       [9.08208890e-01, 9.17911097e-02],
       [2.96918209e-01, 7.03081791e-01],
       [9.93721289e-01, 6.27871111e-03],
       [2.11845657e-01, 7.88154343e-01],
       [9.99722118e-01, 2.77882185e-04],
       [3.80360006e-01, 6.19639994e-01],
       [9.88604736e-01, 1.13952643e-02],
       [2.54520978e-01, 7.45479022e-01],
       [2.97695814e-03, 9.97023042e-01],
       [4.08464022e-01, 5.91535978e-01],
       [6.04340350e-01, 3.95659650e-01],
       [1.92384212e-01, 8.07615788e-01],
       [7.43651986e-01, 2.56348014e-01],
       [1.764322

In [143]:
#Ensemble results....

X_test_list = np.array(X_test)
X_odds = list(zip(X_test_list[:, -2], X_test_list[:, -1], ensemble_probs[:, 0], ensemble_probs[:, 1], y_test))
ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
#display(ev_prepped_df)
get_ev_from_df(ev_prepped_df, True)


          Number of matches: 60
          Number of bets: 58
          Number of winning bets: 34
          Number of losing bets: 24
          Number of underdog bets: 31
          Number of underdog wins: 13
          Number of underdog losses: 18
          Number of Favorite bets: 27
          Number of favorite wins: 21
          Number of favorite losses: 6
          Number of even bets: 0
          Number of even wins: 0
          Number of even losses: 0
          Profit: 676.3539866949042
          Profit per bet: 11.661275632670762
          Profit per match: 11.27256644491507
          
          


0